In [74]:
# import des packages nécessaires
import numpy as np
import pandas as pd

Un système de recommandation est un système intelligent qui essaye deprédire les préférences des utilisateurs sur certains produits. L' objectif est de trouver une relation entre l'utilisateur et les produits afin de maximiser l'engagement utilisateur-produit.

L'application principale des systèmes de recommandation consiste à suggérer une vidéo (une chanson, un film, ...) à l'utilisateur associée aux éléments aimés dans le passé et qui répond à ses gouts.
Ils sont également utilisé pour recommander des contenus en fonction des comportements des utilisateurs sur les réseaux sociaux et les sites d'actualités.

Les deux approches basiques pour créer des systèmes de recommandation à l'aide du machine learning sont les suivants:
- Filtrage Collaboratif : L'hypothèse de cette approche est que les personnes qui ont aimé un article dans le passé aimeront également le même à l'avenir. Cette approche construit un modèle basé sur le comportement passé des utilisateurs. Le comportement de l'utilisateur peut inclure des vidéos visionnées précédemment, des articles achetés, des évaluations données sur des articles. De cette manière, le modèle trouve une association entre les utilisateurs et les éléments. Le modèle est ensuite utilisé pour prédire l'élément susceptible d'intéresser l'utilisateur. La décomposition en valeurs singulières (SVD) est une des techniques utilisée comme approche de filtrage collaboratif dans les systèmes de recommandation.

- Filtrage basé sur le contenu : cette démarche s’appuie sur une description de l’article et un enregistrement des préférences des utilisateurs. Il utilise une séquence de caractéristiques discrètes et pré-étiquetées d'un article afin de recommander des articles supplémentaires avec des propriétés similaires. Cette approche est la mieux adaptée lorsqu'il existe suffisamment d'informations disponibles sur les éléments, mais pas sur les préférences des utilisateurs.

Dans ce TP, nous allons nous concentrer sur la construction d'un système de recommendation de films de type filtrage collaboratif à l'aide de la technique d'algèbre linéaire SVD.

# Lecture données

Téléchargez le dataset "MovieLens 1M movie ratings": https://grouplens.org/datasets/movielens/1m/ . Il s'agit d'un dataset contenant les notes de 1 à 5 données par différents utilisateurs à des films de type différent.

Ensuite importez le ici à l'aide de Pandas. Ce dataset ne demande pas de prétraitement ni de préparation des données.

In [75]:
# TODO inserez le path vers les données
from codecs import utf_8_decode


rating_data_path = 'ml-1m/ratings.dat'
movie_data_path = 'ml-1m/movies.dat'

In [76]:
rating_data = pd.io.parsers.read_csv(rating_data_path, 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')

In [77]:
movie_data = pd.io.parsers.read_csv(movie_data_path,
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::', encoding='latin-1')

In [78]:
# TODO visualisez des exemples du dataset rating_data

display(rating_data.head())

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [79]:
# TODO visualisez des exemples du dataset movie_data

display(movie_data)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


TODO Optionnel : analysez le dataset et visualisez ses caractéristiques et des stats simples liées à son contenu:
- quel est le film le plus populaire en moyenne?
- quels sont les films les plus notés?
- quelle est la note moyenne des films?
- quel est l'utilisateur qui a noté le plus de films? combien?
- ...

In [80]:
print('Films les pluus populaires : ')
rating_data.groupby(['movie_id']).mean().sort_values(by=['rating'], ascending=False)

Films les pluus populaires : 


,user_id,rating,time
movie_id,,,
989,1915.000000,5.0,9.746939e+08
3881,2885.000000,5.0,9.724529e+08
1830,2869.000000,5.0,9.724390e+08
3382,5334.000000,5.0,9.607962e+08
787,1948.666667,5.0,9.741198e+08
...,...,...,...
826,87.000000,1.0,9.776944e+08
3228,2864.000000,1.0,9.763612e+08
2845,5780.000000,1.0,9.581531e+08


In [81]:
print('Films les mieux notés : ')
rating_data.groupby(['movie_id']).count().sort_values(by=['user_id'], ascending=False)

Films les mieux notés : 


,user_id,rating,time
movie_id,,,
2858,3428,3428,3428
260,2991,2991,2991
1196,2990,2990,2990
1210,2883,2883,2883
480,2672,2672,2672
...,...,...,...
3237,1,1,1
763,1,1,1
624,1,1,1


In [82]:
print('Note moyenne des films : ' + str(rating_data.rating.values.mean()))

Note moyenne des films : 3.581564453029317


In [83]:
print('Utilisateur ayant noté le plus de films : ')
rating_data.groupby(['user_id']).count().sort_values(by=['movie_id'], ascending=False).head(1)

Utilisateur ayant noté le plus de films : 


,movie_id,rating,time
user_id,,,
4169,2314,2314,2314


# Modèle

La décomposition en valeurs singulières (SVD) est une méthode d'algèbre linéaire qui est utilisée comme technique de réduction de dimensionnalité dans l'apprentissage automatique (et dans d'autres domaines).
La SVD est une technique de factorisation matricielle, qui décompose une matrice en d'autres matrices plus simples, et elle est utilisée pour réduire le nombre de features d'un ensemble de données en réduisant la dimension spatiale de la dimension $N$ à la dimension $K$.

Dans le cadre des système de recommandation, la SVD est utilisé comme technique de filtrage collaboratif.
Elle se base sur l'utilisation d'une matrice $A$ où chaque ligne représente un utilisateur et chaque colonne représente un élément (film, livre, produit, ...). Les valeurs de cette matrice sont les notes attribuées aux éléments par les utilisateurs.


La matrice $A$ est factorisée à l'aide de la SVD:
$A = U S V^T$

- $U$ de taille représente la relation entre les utilisateurs et les facteurs latents du nouvel espace vectoriel
- $V$ représente la relation entre les éléments et les facteurs latents du nouvel espace vectoriel
- $S$ décrit l'intensité de chaque facteur latent

Dans la SVD, les colonnes de la matrice $U$ sont des vecteurs propres de $A A^T$ et les lignes de la matrice $V$ sont des vecteurs propres de $A^T A$. Ce qui est intéressant, c'est que $A^T A$ et $A A^T$ sont potentiellement de taille différente (car la matrice $A$ peut être de forme non carrée), mais ils ont le même ensemble de valeurs propres, qui sont le carré des valeurs sur la diagonale de $S$.
C'est pourquoi le résultat de la SVD peut révéler beaucoup de choses sur la matrice $A$.

La SVD réduit la dimension de la matrice $A$ en extrayant ses facteurs latents et en gardans seulement ceux avec importantece éléveé. Les facteurs latents ici représentent des caractéristiques des éléments, par exemple, le genre de musique ou de film, mais parfois ils sont un peu complèxes à interpréter par un humain. 
Cette opération cartographie chaque utilisateur et chaque élément dans un espace latent à $K$ dimensions. Ce mappage facilite une représentation claire des relations entre les utilisateurs et les éléments.

Imaginez que nous ayons collecté dans $A$ des notes de films de manière que les films sont des colonnes et les utilisateurs sont des lignes, et les éléments de la matrice sont les notes qu'une personne a données à un film.
Dans ce cas, $A A^T$ est un tableau personne-personne, où l'on peut retrouver les liens de similarité entre les notes de différentes personnes. 
De même, $A^T A$ est un tableau film-film dont les éléments contiennet la similarité des notes entre films différents.

In [84]:
# fonction pour trouver l'id d'un film sachant son titre en anglais
# fonctionne eseulement si le titre est dans la base de données

def find_id_given_title(given_title):
    # match title with the one in the database
    matching_title = np.unique([title for title in movie_data.title if given_title in title])[0]
    # find id number
    return movie_data[movie_data.title == matching_title].movie_id.values[0]

In [85]:
# fonction qui calcule la cosine similarity entre un film donnée et les autres du dataset
# et qui donne en sortie les id des 10 films les plus similaires sur la base des notes des utilisateurs.
# cette similarité peut être calculée sur les features extraites par la SVD (les vecteurs du nouvel espace V)

def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[1:top_n+1]

In [86]:
# fonction pour imprimer les titres des top 10 films les plus similaires à un film donné

def print_similar_movies(movie_data, movie_id, V, top_n=10, k=50):
    
    sliced = V.T[:, :k] # utilisation seulement des K features latentes les plus importantes
    top_indexes = top_cosine_similarity(sliced, movie_id, top_n)
    
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

Création de la matrice $A$

In [118]:
# TODO créez la matrice ratings_mat contenant les notes des films par utilisateur
# (films sur les lignes, utilisateurs sur les colonnes)

# ratings_mat = pd.pivot_table(rating_data, values='rating', index='movie_id', columns='user_id')
# ratings_mat.fillna(0, inplace=True)
# ratings_mat = ratings_mat.values

ratings_mat = np.ndarray(shape=(np.max(rating_data.movie_id.values), np.max(rating_data.user_id.values)), dtype=float)
ratings_mat[rating_data.movie_id.values-1, rating_data.user_id.values-1] = rating_data.rating.values
print(ratings_mat)

[[5. 0. 0. ... 0. 0. 3.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [119]:
# normalisation de la matrice : on soustrait la moyenne
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T
# ultérieure normalisation et transposition pour passer à la matrice A "classique"
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)

print(A)

[[ 0.05685934 -0.00591061 -0.00379817 ... -0.00052152 -0.0004109
  -0.00386402]
 [-0.02268632 -0.00591061 -0.00379817 ... -0.00052152 -0.0004109
  -0.00386402]
 [-0.02268632 -0.00591061 -0.00379817 ... -0.00052152 -0.0004109
  -0.00386402]
 ...
 [-0.02268632 -0.00591061 -0.00379817 ... -0.00052152 -0.0004109
  -0.00386402]
 [-0.02268632 -0.00591061 -0.00379817 ... -0.00052152 -0.0004109
  -0.00386402]
 [ 0.02504108 -0.00591061 -0.00379817 ... -0.00052152 -0.0004109
  -0.00386402]]


Calcul de la SVD avec numpy

In [120]:
# TODO calculez la Singular Value Decomposition (SVD) à l'aide de numpy

U, S, Vh = np.linalg.svd(A, full_matrices=True)

In [121]:
# TODO : visualisez la taille des matrices U, S et Vh. Qu'est-ce que vous pouvez conclure? 

print(f'U shape: {U.shape}\n'
      f'S shape: {S.shape}\n'
      f'Vh shape: {Vh.shape}\n')


def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))


def similarity(v):
    search_similar = v
    highest_similarity = -np.inf
    highest_sim_col = -1

    for col in range(1,Vh.shape[1]):
        similarity = cosine_similarity(Vh[:,search_similar], Vh[:,col])
        
        if similarity > highest_similarity:
            highest_similarity = similarity
            highest_sim_col = col
            
    return print("Column %d is most similar to column %s" %(highest_sim_col, search_similar))

similarity(0)

U shape: (6040, 6040)
S shape: (3952,)
Vh shape: (3952, 3952)

Column 2354 is most similar to column 0


utilisation de la SVD pour la recommendation de films

In [122]:
movie_data.head(20)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [125]:
# TODO imprimer les 10 films les plus similaires à recommender à quelqu'un qui a aimé un film de votre choix
# à l'aide des fontions fournies

mv = find_id_given_title('Jumanji (1995)')
print_similar_movies(movie_data, mv, Vh, top_n=10, k=50)

Recommendations for Jumanji (1995): 

Hook (1991)
Indian in the Cupboard, The (1995)
NeverEnding Story II: The Next Chapter, The (1990)
Dragonheart (1996)
FairyTale: A True Story (1997)
NeverEnding Story, The (1984)
Santa Clause, The (1994)
Simple Wish, A (1997)
Borrowers, The (1997)
Labyrinth (1986)


C:\Users\BEP29\AppData\Local\Temp\ipykernel_5108\4201844677.py:9: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)


TODO optionnel :  au lieu de calculer les films similaires sur les resultats de la SVD, est-ce que nous pouvons les extraire directement à niveau de la matrice A? si oui, comment? Quels sont les avantages et les desavantages? Essayez

In [ ]:
#   (ㆆ _ ㆆ)

# Références

https://machinelearningmastery.com/using-singular-value-decomposition-to-build-a-recommender-system/

http://snap.stanford.edu/class/cs246-2015/handouts.html